# kneser_ney smoothing

In [2]:
from collections import Counter
import math
import re
import numpy as np
from nltk.util import ngrams

In [3]:
mask_file_path='NLP-01-2-HW1-Data/mask.txt'
mask_gold_file_path='NLP-01-2-HW1-Data/mask_gold.txt'

incomplete_file_path='NLP-01-2-HW1-Data/incomplete.txt'

In [4]:
def load_dataset(path):
    with open(path, 'r') as file:
        txt=file.read()
    return txt

In [5]:
def preprocess_txt(txt):
    '''Preprocess the text by removing any unwanted characters,
     converting all the text to lowercase'''


    # remove any unwanted characters
    txt=txt.replace('\u200c',' ')
    txt=txt.replace('\n',' ')

    txt= re.sub(r'[^\w\s]','',txt)
    # convert all the english text to lowercase
    txt=txt.lower()
    
    return txt

In [6]:
def txt2word(txt):
    # split the text into individual words.
    words= txt.split(' ')
    # remove '_' in case it appears individually as a word
    if "_" in words:
        words.remove("_")
    # delete empty strings
    if "" in words:
        words.remove("")
        
    return words

In [7]:
def txt2sentence(txt):
    # split the text into individual sentence.
    sentences = re.split(';|\.|\n|\?|\؟',txt)
    # delete empty strings
    if '' in sentences:
        sentences.remove('')
    
    return sentences

In [8]:
def generate_trigrams(sentence):
    trigrams=Counter()

    for i in range(2, len(sentence)):
        trigram=tuple(sentence[i-2:i+1])
        trigrams[trigram] += 1
    return trigrams

In [9]:


class KNSmoother:
    
    def __init__(self, trigrams,discount=0.75):
        self.trigram_counts=trigrams
        self.bigrams=self.get_bigrams()
        self.unigrams=self.get_unigrams()
        self.total_count=sum(self.unigrams.values())
        self.discount=discount # Kneser-Ney discount parameter
        
    def get_bigrams(self):
        bigrams=Counter()
        for trigram, count in self.trigram_counts.items():
            bigram=trigram[0:2]
            bigrams[bigram] += count
        return bigrams
    
    def get_unigrams(self):
        unigrams=Counter()
        for trigram, count in self.trigram_counts.items():
            unigram=trigram[0]
            unigrams[unigram] += count
        return unigrams


    
        
    def kn_smooth(self, trigram):
        trigram_count=self.trigram_counts[trigram]
        bigram=trigram[0:2]
        unigram=trigram[0]
        bigram_count=self.bigrams[bigram]
        unigram_count=self.unigrams[unigram]
        # handling out of vocab
        if bigram_count==0:
            total_grams=sum(self.trigram_counts.values()) 
            return self.discount/total_grams
        
        
        # Compute D and lambda values
        D=self.discount * (self.get_num_distinct_trigrams_starting_with_bigram(bigram) / bigram_count)
        lamb=(D / unigram_count) * self.get_num_trigrams_starting_with_bigram(bigram)
        
        # Compute P_kn value
        P_continuation=(self.discount / bigram_count) * self.get_num_distinct_continuations(bigram)
        P_trigram=max(trigram_count - self.discount, 0) / bigram_count
        P_kn=P_trigram + lamb * P_continuation
        
        return P_kn

    
    def get_num_distinct_trigrams_starting_with_bigram(self, bigram):
        num_distinct_trigrams=0
        for trigram in self.trigram_counts:
            if trigram[0:2] == bigram:
                num_distinct_trigrams += 1
        return num_distinct_trigrams
    
    def get_num_trigrams_starting_with_bigram(self, bigram):
        num_trigrams=0
        for trigram in self.trigram_counts:
            if trigram[0:2] == bigram:
                num_trigrams += self.trigram_counts[trigram]
        return num_trigrams
    
    def get_num_distinct_continuations(self, bigram):
        continuations=set()
        for trigram in self.trigram_counts:
            if trigram[0:2] == bigram:
                continuations.add(trigram[2])
        return len(continuations)
    
    def get_kn_smooth(self, sentence):
        probs=[]
        for i in range(2, len(sentence)):
            trigram=tuple(sentence[i-2:i+1])
            P_kn=self.kn_smooth(trigram)
            probs.append(P_kn)
        return probs


    def score_sentence(self, sentence):
        score=0.0
        for i in range(2, len(sentence)):
            trigram=tuple(sentence[i-2:i+1])
            P_kn=self.kn_smooth(trigram)
            # print(math.log(P_kn, 2))
            score += math.log(P_kn, 2)
        return score
    
    def perplexity(self, corpus):
        logprob=0
        word_count=0
        for sentence in corpus:
            logprob += self.score_sentence(sentence)
            word_count += len(sentence) - 2
        logprob /= word_count
        perplexity=2**(-logprob)
        return perplexity

In [10]:
# test_words

In [11]:
#load and preprocess train data
train_txt=load_dataset("NLP-01-2-HW1-Data/train.txt")
train_txt=preprocess_txt(train_txt)
train_words=txt2word(train_txt)
trigram_words=generate_trigrams(train_words)


In [12]:
valid_courpus=load_dataset("NLP-01-2-HW1-Data/valid.txt")
preprocess_valid_courpus=[preprocess_txt(sentence) for sentence in txt2sentence(valid_courpus)]
valid_words=[txt2word(sentence) for sentence in preprocess_valid_courpus]


In [13]:
len(valid_words)

2128

In [15]:
# resualts=[]
# for  d in  [0.5, 0.7, 0.9]:
#     kn_smoother=KNSmoother(trigram_words)

#     perplexity=kn_smoother.perplexity(valid_words[:])
#     print(f"discount:{d},Perplexity: {perplexity}")
#     resualts.append((d,perplexity))

In [16]:
mask_sentence=[]
with open(mask_file_path) as file_in:
    for line in file_in:
       
        words=(re.split('\u200c| ',(line.split('\t')[1])))
        for j in range(words.count('#')):
            if j==0:
                idx=0
                pre_idx=0
            else:
                pre_idx=idx+1
            idx=words.index('#',idx+1)
            # print(words[pre_idx:idx],j)
            mask_sentence.append(' '.join(words[pre_idx:idx]))
        

In [ ]:
incomplete_data=[]
with open(incomplete_file_path) as file_in:
    for line in file_in:
        words=(re.split('\u200c| ',preprocess_txt(line.split('\t')[1])))
        incomplete_data.append(' '.join(words))
        

In [ ]:
vocab=list(set(train_words))
# vocab.remove('')
# vocab

In [19]:
test_courpus=load_dataset("NLP-01-2-HW1-Data/test.txt")
preprocess_test_courpus=[preprocess_txt(sentence) for sentence in txt2sentence(test_courpus)]
test_words=[txt2word(sentence) for sentence in preprocess_test_courpus]


In [17]:
kn_smoother=KNSmoother(trigram_words)


In [ ]:
kn_smoother.kn_smooth('پیشرفت را به')

1.9849619284302127e-06

In [ ]:
# kn_smoother.get_kn_smooth()

In [ ]:
logprob=kn_smoother.score_sentence(test_words[0])
print(f"Log probability: {logprob}")

Log probability: -456.34306508166765


In [21]:
perplexity=kn_smoother.perplexity(test_words)
print(f"Perplexity: {perplexity}")


KeyboardInterrupt: 

In [ ]:
def predict_next_word(sentence,kn_smoother,vocab):
    words = sentence.split()
    if len(words) < 2:
        # print('small')
        return None
    elif len(words) == 2:
        bi = (words[0], words[1])
        tri_probs = [kn_smoother.kn_smooth(' '.join([bi[0], bi[1], word])) for word in vocab]
        pred= vocab[tri_probs.index(max(tri_probs))]
    else:
        tri = (words[-3], words[-2], words[-1])
        bi = (words[-2], words[-1])
        tri_probs = [kn_smoother.kn_smooth(' '.join([tri[0], tri[1], word])) for word in vocab]
        bi_probs = [kn_smoother.kn_smooth(' '.join([bi[0], bi[1], word])) for word in vocab]
        probs = [p_tri * p_bi for p_tri, p_bi in zip(tri_probs, bi_probs)]
        pred= vocab[probs.index(max(probs))]
    # if pred=='':
    #     idx=np.array(probs).argsort()[-2]
    #     pred=vocab[idx]
        # print(max(probs))

    return pred


In [ ]:
words = sentence.split()
bi = (words[0], words[1])
[kn_smoother.kn_smooth(' '.join([bi[0], bi[1], word])) for word in vocab]

'عمومی ترین'

In [ ]:
predicted_word = predict_next_word(mask_sentence[5],kn_smoother,vocab)
if predicted_word=='':
    print('predicted_word:','Null')
else:
    print('predicted_word:',predicted_word)
print(' '.join([mask_sentence[5],predicted_word]))
print("_______________________")

predicted_word: Null
بر دشمنان خود 
_______________________


In [ ]:
for sentence in mask_sentence[6:]:
    # print(sentence)
    predicted_word = predict_next_word(sentence,kn_smoother,vocab)
    if predicted_word=='':
        print('predicted_word:','Null')
    else:
        print('predicted_word:',predicted_word)
    print(' '.join([sentence,predicted_word]))
    print("_______________________")

predicted_word: Null
عمومی ترین 
_______________________
predicted_word: Null
جانوران که طی مراحل 
_______________________
predicted_word: Null
نیز پیش از همه 
_______________________


In [ ]:
for sentence in mask_sentence:
    predicted_word = predict_next_word(sentence,kn_smoother,vocab)
    if predicted_word=='':
        print('predicted_word:','Null')
    else:
        print('predicted_word:',predicted_word)
    print(' '.join([sentence,predicted_word]))
    print("_______________________")

predicted_word: Null
سارتر به عنوان روشن فکری فعال از نظر 
_______________________
predicted_word: Null
این تیم در سال ۱۸۹۹ 
_______________________
predicted_word: Null
شد و تا به حال موفق به کسب یک عنوان 
_______________________
predicted_word: Null
بزرگترین کلیسای 
_______________________
predicted_word: Null
کریم خان پس از 
_______________________


KeyboardInterrupt: 

In [ ]:
for sentence in incomplete_data:
    predicted_word = predict_next_word(sentence,kn_smoother,vocab)
    if predicted_word=='':
        print('predicted_word:','Null')
    else:
        print('predicted_word:',predicted_word)
    print(' '.join([sentence,predicted_word]))
    print("_______________________")

predicted_word: Null
در جریان انقلاب مشروطه ابتدا به عنوان یکی از نیروهای محمدعلی شاه با   
_______________________
predicted_word: Null
شرکت خدمات مالی و  
_______________________
predicted_word: Null
شخص موسی تورات را   
_______________________
predicted_word: Null
نام امازون را از یک لغت نامه   
_______________________
predicted_word: Null
تیم سپاهان اصفهان که در ابتدا شعبه  
_______________________


In [ ]:
# sentence='ساخت این شهر به این علت بود که می‌خواستند محل پایتخت برزیل که پیش از آن شهر ریو دوژانیرو بود را تغییر داده و بیشتر به مرکز و غرب کشور بیاورند تا پیشرفت را به مناطق داخلی‌تر برزیل توسعه دهد.'
# sentence=preprocess_txt(sentence)

In [ ]:
# words=txt2word(sentence)
# kn_smoother=KNSmoother(trigram_words)
# print(kn_smoother.get_kn_smooth(words))
# logprob=kn_smoother.score_sentence(words)
# perplexity=kn_smoother.perplexity([words])
# print(f"Log probability: {logprob}")
# print(f"Perplexity: {perplexity}")

In [ ]:
# train_txt='the quick brown fox jumps over the lazy dog the lazy dog is not very quick the quick brown fox is very quick indeed'
# words=txt2word(train_txt)
# trigram_words=generate_trigrams(words)

In [ ]:
# kn_smoother=KNSmoother(trigram_words)
# sentence='the quick brown fox jumps'
# words=txt2word(sentence)
# kn_smoother.get_kn_smooth(words)

In [ ]:

# # Evaluate the language model on a test sentence
# test_sentence=['the quick brown fox jumps over the lazy dog']
# test_sentence=txt2word(sentence)
# logprob=kn_smoother.score_sentence(test_sentence)
# perplexity=kn_smoother.perplexity([test_sentence])
# print(f"Log probability: {logprob}")
# print(f"Perplexity: {perplexity}")